# 2021/11/11
# 本代码计算气压梯度力
# merra2资料与famil实验的，MERRA2资料分为气候态、早年、晚年

In [1]:
import os
import sys
import xarray as xr
import numpy as np
module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *

In [2]:
path0     =  '/home/sun/qomo-data/zhuang_plev/'
exp_name  =  ['con','id']

f0        =  xr.open_dataset(path0+'plev_'+exp_name[0]+'_Z3.nc')
f1        =  xr.open_dataset(path0+'plev_'+exp_name[1]+'_Z3.nc')

In [3]:
# 读入数据
con_z3    =  f0.Z3.data*9.8
id_z3     =  f1.Z3.data*9.8

# 计算y方向梯度力

'''获取地理坐标'''
disy,disx,location =   cal_xydistance(f0.lat,f0.lon)

'''计算梯度'''
gradient_y_con     =   np.gradient(con_z3,location,axis=2)
gradient_y_id      =   np.gradient(id_z3,location,axis=2)

In [4]:
ncfile  =  xr.Dataset(
    {
        "gradient_y_con": (["time", "lev", "lat", "lon"], gradient_y_con),
        "gradient_y_id": (["time", "lev", "lat", "lon"], gradient_y_id),
    },
    coords={
        "lon": (["lon"], f0.lon.data),
        "lat": (["lat"], f0.lat.data),
        "lev": (["lev"], f0.lev.data),
        "time": (["time"], np.linspace(1,365,365)),
    },
)
ncfile.gradient_y_con.attrs  =  f0.Z3.attrs
ncfile.gradient_y_id.attrs   =  f0.Z3.attrs
ncfile["lat"].attrs  =  f0["lat"].attrs
ncfile["lon"].attrs  =  f0["lon"].attrs
ncfile["lev"].attrs  =  f0["lev"].attrs
ncfile.to_netcdf("/home/sun/data/geopotential_height_gradient_con_id.nc")

In [5]:
# 处理成候平均
f1  =  xr.open_dataset('/home/sun/data/geopotential_height_gradient_con_id.nc')
pentad_gradient_y_con  =  np.zeros((73,f1.gradient_y_con.shape[1],f1.gradient_y_con.shape[2],f1.gradient_y_con.shape[3]))
pentad_gradient_y_id   =  pentad_gradient_y_con.copy()

for i in range(0,73):
    pentad_gradient_y_con[i,:]  =  np.average(f1.gradient_y_con[i*5:i*5+5,:],axis=0)
    pentad_gradient_y_id[i,:]   =  np.average(f1.gradient_y_id[i*5:i*5+5,:],axis=0)

ncfile  =  xr.Dataset(
    {
        "pentad_gradient_y_con": (["time", "lev", "lat", "lon"], pentad_gradient_y_con),
        "pentad_gradient_y_id": (["time", "lev", "lat", "lon"],  pentad_gradient_y_id),
    },
    coords={
        "lon": (["lon"], f0.lon.data),
        "lat": (["lat"], f0.lat.data),
        "lev": (["lev"], f0.lev.data),
        "time": (["time"], np.linspace(1,73,73)),
    },
)
ncfile.pentad_gradient_y_con.attrs  =  f0.Z3.attrs
ncfile.pentad_gradient_y_id.attrs   =  f0.Z3.attrs
ncfile["lat"].attrs  =  f0["lat"].attrs
ncfile["lon"].attrs  =  f0["lon"].attrs
ncfile["lev"].attrs  =  f0["lev"].attrs
ncfile.to_netcdf("/home/sun/data/pentad_geopotential_height_gradient_con_id.nc")